In [ ]:
import numpy as np
import holoviews as hv
import dask.dataframe as dd

from holoviews import opts
from holoviews.operation.datashader import aggregate

renderer = hv.renderer('bokeh')

# Set plot and style options
opts.defaults(
    opts.Curve(xaxis=None, yaxis=None, show_grid=False, show_frame=False,
               color='orangered', framewise=True, width=100),
    opts.Image(width=800, height=400, shared_axes=False, logz=True,
               xaxis=None, yaxis=None, axiswise=True),
    opts.HLine(color='white', line_width=1),
    opts.Layout(shared_axes=False),
    opts.VLine(color='white', line_width=1))

# Read the parquet file
df = dd.read_parquet('./data/nyc_taxi_wide.parq').persist()

# Declare points
points = hv.Points(df, kdims=['pickup_x', 'pickup_y'], vdims=[])

# Use datashader to rasterize and linked streams for interactivity
agg = aggregate(points, link_inputs=True, x_sampling=0.0001, y_sampling=0.0001)
pointerx = hv.streams.PointerX(x=np.mean(points.range('pickup_x')), source=points)
pointery = hv.streams.PointerY(y=np.mean(points.range('pickup_y')), source=points)
vline = hv.DynamicMap(lambda x: hv.VLine(x), streams=[pointerx])
hline = hv.DynamicMap(lambda y: hv.HLine(y), streams=[pointery])

sampled = hv.util.Dynamic(agg, operation=lambda obj, x: obj.sample(pickup_x=x),
                          streams=[pointerx], link_inputs=False)

hvobj = ((agg * hline * vline) << sampled)

# Obtain Bokeh document and set the title
doc = renderer.server_doc(hvobj)
doc.title = 'NYC Taxi Crosshair'

In [9]:

import holoviews as hv
from holoviews import opts
hv.extension('bokeh')


from bokeh.sampledata.us_counties import data as counties
from bokeh.sampledata.unemployment import data as unemployment



counties = [dict(county, Unemployment=unemployment[cid])
            for cid, county in counties.items()
            if county["state"] == "tx"]

choropleth = hv.Polygons(counties, ['lons', 'lats'], [('detailed name', 'County'), 'Unemployment'])

choropleth.opts(
    opts.Polygons(logz=True, tools=['hover'], xaxis=None, yaxis=None,
                   show_grid=False, show_frame=False, width=500, height=500,
                   color_index='Unemployment', colorbar=True, toolbar='above', line_color='white'))

ImportError: None of the backends could be imported